# Train a dataset from Interface 2018/12 with Keras

- Unlike small book image dataset, it was little bit harder to fine-tune parameters.
- Similar accuracy with fast.ai could be achieved, but spent a lot more effort.

Using fast.ai library would be the shortest path to reach the goal.

In [100]:
##### import warnings
warnings.simplefilter('ignore')
import numpy as np
np.warnings.filterwarnings('ignore')
np.random.seed(1001)

import os
import sys
import shutil
from pathlib import Path

import IPython
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm_notebook
from sklearn.model_selection import StratifiedKFold

matplotlib.style.use('ggplot')
%matplotlib inline

from keras.applications.vgg16 import VGG16

from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, AveragePooling2D, Conv2D, Softmax
from keras import backend as K
import keras

from skimage.io import imread
from scipy.misc import imresize
from keras.preprocessing.image import ImageDataGenerator
from mixup_generator import MixupGenerator
from random_eraser import get_random_eraser
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)

### Model
def model_imagenet_x(input_shape, num_classes, weights='imagenet', show_layers=False):
    # create the base pre-trained model
    #base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model = VGG16(weights=weights, include_top=False, input_shape=input_shape)

    # add a global spatial average pooling layer & FC
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(256, activation='relu')(x)

    if show_layers:
        for i, layer in enumerate(base_model.layers):
            print(i, layer.name)

    predictions = Dense(num_classes, activation='softmax')(x)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    return model, base_model

def model_imagenet_x_fastailike(input_shape, num_classes, weights='imagenet', show_layers=False):
    # create the base pre-trained model
    #base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model = VGG16(weights=weights, include_top=False, input_shape=input_shape)

    # add a global spatial average pooling layer & FC
    x = base_model.output
    x = Conv2D(num_classes, kernel_size=3, padding='valid')(x)
    x = AveragePooling2D(pool_size=5)(x)
    x = Flatten()(x)
    predictions = Softmax()(x)

    if show_layers:
        for i, layer in enumerate(base_model.layers):
            print(i, layer.name)

    # this is the model we will train
    model = Model(inputs=base_model.input, outputs=predictions)

    return model, base_model

### Configuration management class
class Config:
    def __init__(self,
                 learning_rate=0.0001,
                 batch_size=16,
                 shape=[224, 224, 3],
                 use_mixup=True,
                 use_augmentations=True,
                verbose=1):
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.shape = shape
        self.verbose = verbose
        self.use_mixup = use_mixup
        self.use_augmentations = use_augmentations

### Dataset distribution utility
def get_class_distribution(y):
    # y_cls can be one of [OH label, index of class, class label name]
    # convert OH to index of class
    y_cls = [np.argmax(one) for one in y] if len(np.array(y).shape) == 2 else y
    # y_cls can be one of [index of class, class label name]
    classset = sorted(list(set(y_cls)))
    sample_distribution = {cur_cls:len([one for one in y_cls if one == cur_cls]) for cur_cls in classset}
    return sample_distribution

def get_class_distribution_list(y, num_classes):
    dist = get_class_distribution(y)
    assert(y[0].__class__ != str) # class index or class OH label only
    list_dist = np.zeros((num_classes))
    for i in range(num_classes):
        if i in dist:
            list_dist[i] = dist[i]
    return list_dist

from imblearn.over_sampling import RandomOverSampler
def balance_class_by_over_sampling(X, y): # Naive: all sample has equal weights
    Xidx = [[xidx] for xidx in range(len(X))]
    y_cls = [np.argmax(one) for one in y]
    classset = sorted(list(set(y_cls)))
    sample_distribution = [len([one for one in y_cls if one == cur_cls]) for cur_cls in classset]
    nsamples = np.max(sample_distribution)
    flat_ratio = {cls:nsamples for cls in classset}
    Xidx_resampled, y_cls_resampled = RandomOverSampler(ratio=flat_ratio, random_state=42).fit_sample(Xidx, y_cls)
    sampled_index = [idx[0] for idx in Xidx_resampled]
    return np.array([X[idx] for idx in sampled_index]), np.array([y[idx] for idx in sampled_index])

### Dataset management class
class LabeledDataset:
    image_suffix = ['.jpg', '.JPG']
    def __init__(self, datapath, shape, batch_size):
        self.datapath = Path(datapath)
        self.shape = shape
        self.batch_size = batch_size
    def load_image(filename, shape, rescale_factor):
        img = imread(filename)
        return imresize(img, shape[:2]) * rescale_factor
    def load_as_image(self):
        # datapath shall contain label/file labeled data files
        train_files = sorted([x for x in self.datapath.glob('*/*') if x.suffix in LabeledDataset.image_suffix])
        self.X_train = np.array([LabeledDataset.load_image(filename, self.shape, rescale_factor=1/255.)
                                    for filename in train_files])
        y_train_label = [filename.parent.name for filename in train_files]
        self.labels = sorted(list(set(y_train_label)))
        self.label2int = {label:i for i, label in enumerate(self.labels)}
        self.y_train = to_categorical([self.label2int[label] for label in y_train_label])
    def split_train_valid(self, test_size=0.2, random_state=42):
        self.cur_X_train, self.cur_X_valid, self.cur_y_train, self.cur_y_valid = train_test_split(
            self.X_train, 
            self.y_train, 
            test_size=test_size,
            random_state=random_state)
        self.cur_X_train, self.cur_y_train = \
            balance_class_by_over_sampling(self.cur_X_train, self.cur_y_train)
    def load_test_as_image(self, test_datapath):
        test_datapath = Path(test_datapath)
        test_files = sorted([x for x in test_datapath.glob('*') if x.suffix in LabeledDataset.image_suffix])
        self.X_test = np.array([LabeledDataset.load_image(filename, self.shape, rescale_factor=1/255.)
                                for filename in test_files])
    def create_test_generator(self, IDG_options={}):
        test_datagen = ImageDataGenerator(**IDG_options)
        y_test_dummy = to_categorical([0 for _ in range(len(self.X_test))])
        self.test_gen = test_datagen.flow(self.X_test, y_test_dummy,
                                          batch_size=len(self.X_test), shuffle=False) ########## self.batch_size
    def create_generator(self, conf, IDG_options={}):
        aug_datagen = ImageDataGenerator(**IDG_options)
        if conf.use_mixup:
            self.train_gen = MixupGenerator(self.cur_X_train, self.cur_y_train, 
                                            alpha=1.0, batch_size=self.batch_size,
                                            datagen=aug_datagen)()
        else:
            self.train_gen = aug_datagen.flow(self.cur_X_train, self.cur_y_train, 
                                              batch_size=self.batch_size)
        plain_datagen = ImageDataGenerator()
        self.valid_gen = plain_datagen.flow(self.cur_X_valid, self.cur_y_valid,
                                                              batch_size=self.batch_size, shuffle=False)
    def train_steps_per_epoch(self):
        return len(self.cur_X_train) // self.batch_size
    def valid_steps_per_epoch(self):
        return len(self.cur_X_valid) // self.batch_size

def reset_generator():
    if conf.use_augmentations:
        IDG_options={'horizontal_flip': True, 'vertical_flip': True,
                     'rotation_range': 40, 'zoom_range': 0.2,
                      'preprocessing_function': get_random_eraser(v_l=np.min(d.X_train),                                                           
                                                    v_h=np.max(d.X_train))}
    else:
        IDG_options={}
    d.create_generator(conf, IDG_options)

In [4]:
! wget https://github.com/yasudakn/umaibar/raw/master/interface-201812-umaibar-data-content.zip
! mkdir -p interface-201812-umaibar-data-content
! unzip interface-201812-umaibar-data-content.zip
! mv data-content interface-201812-umaibar-data-content
! rm interface-201812-umaibar-data-content.zip

--2018-10-25 17:06:59--  https://github.com/yasudakn/umaibar/raw/master/interface-201812-umaibar-data-content.zip
Resolving github.com (github.com)... 192.30.255.112, 192.30.255.113
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/yasudakn/umaibar/master/interface-201812-umaibar-data-content.zip [following]
--2018-10-25 17:07:00--  https://raw.githubusercontent.com/yasudakn/umaibar/master/interface-201812-umaibar-data-content.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.88.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.88.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21043355 (20M) [application/zip]
Saving to: ‘interface-201812-umaibar-data-content.zip.1’

interface-201812-um 100%[===================>]  20.07M  4.17MB/s    in 4.8s    

2018-10-25 17:07:06 (4.16 MB/s) - ‘int

In [36]:
DATAROOT = 'interface-201812-umaibar-data-content/data-content'
datapath = Path(DATAROOT)

In [37]:
conf = Config(batch_size=8, shape=[224, 224, 3], 
              use_mixup=True, use_augmentations=True)

d = LabeledDataset(datapath / 'train', conf.shape, conf.batch_size)
d.load_as_image()
d.split_train_valid()

# ImageNet Based

In [55]:
conf.learning_rate

0.0001

In [101]:
def imagenet_based_evaluation(train_points, model_footer, epochs):
    for train_point in train_points:
        weight_filename = 'model_weights_%s_imagenet_%s.h5' % (train_point, model_footer)
        callbacks = [
            ModelCheckpoint(weight_filename, monitor='val_loss', verbose=2, save_best_only=True, save_weights_only=True),
            #TensorBoard(log_dir=d.file_with_prefix('logs%s/fold_%d' % (conf.prefix, i)), write_graph=True)
        ]
        reset_generator()

        print('[%s]' % train_point)
        model, base_model = model_imagenet_x(input_shape=conf.shape, num_classes=len(d.labels),
                                             weights='imagenet')
        for layer in base_model.layers:
            layer.trainable = False
        model.compile(loss='categorical_crossentropy',
                      optimizer=keras.optimizers.Adam(lr=0.001),#, decay=1e-7, epsilon=1e-8),#conf.learning_rate),
                      metrics=['accuracy'])
        model.summary()
        model.fit_generator(d.train_gen, 
                  steps_per_epoch=d.train_steps_per_epoch(),
                  epochs=15,
                  validation_data=d.valid_gen,
                  validation_steps=d.valid_steps_per_epoch(),
                  callbacks=callbacks,
                  verbose=conf.verbose)
        print('\n----------------------------------------------')
        print('Roughly trained, now fine tune from', train_point, 'for', epochs, 'epochs.')
        trainable = False
        for layer in base_model.layers:
            if layer.name == train_point:
                trainable = True
            layer.trainable = trainable
        model.load_weights(weight_filename)
        model.compile(loss='categorical_crossentropy',
#                      optimizer=keras.optimizers.Adam(lr=0.00001, decay=1e-7, epsilon=1e-8), # not so much good... 0.9 or so
#                      optimizer=keras.optimizers.Adam(lr=0.00001), # 0.92
#                      optimizer=keras.optimizers.Adam(lr=0.00001, decay=1e-4), # 0.9
                      optimizer=keras.optimizers.RMSprop(lr=0.00001), # 0.92
#                      optimizer=keras.optimizers.SGD(lr=0.0002, momentum=0.9, decay=1e-7), # 0.92
#                      optimizer=keras.optimizers.Adam(lr=0.00001, decay=1e-7), # 0.9
                      metrics=['accuracy'])
        history = model.fit_generator(d.train_gen, 
                  steps_per_epoch=d.train_steps_per_epoch(),
                  epochs=epochs,
                  validation_data=d.valid_gen,
                  validation_steps=d.valid_steps_per_epoch(),
                  callbacks=callbacks,
                  verbose=conf.verbose)

In [ ]:
conf = Config(batch_size=8, shape=[224, 224, 3], 
              use_mixup=True, use_augmentations=True)
imagenet_based_evaluation(train_points=['block3_conv1'], model_footer='full_aug', epochs=100)

# TBD - Visualization

Followings are NOT executed, left just for the future attempts.

In [104]:
model, base_model = model_imagenet_x(input_shape=conf.shape, num_classes=len(d.labels),
                                     weights='imagenet')
model.load_weights('model_weights_block3_conv1_imagenet_full_aug.h5')

In [8]:
import cv2

def imshow_friendly(img):
    img_temp = img - np.min(img)
    img_temp = img_temp/np.max(img_temp)
    friendly = np.uint8(255 * img_temp)
    return friendly

def visualize_cam(model, model_weight, test_file_index, datapath, 
                  expected_preds, test_time_aug_param={}):
    d.load_test_as_image(datapath)
    d.create_test_generator(test_time_aug_param)
    model.load_weights(model_weight)
    last_conv_layer = model.get_layer('block5_conv3')
    cur_X_test, cur_y_test = next(d.test_gen)
    x = np.array([cur_X_test[test_file_index]])
    preds = model.predict(x)
    targ_class = np.argmax(preds[0])
    result = calc_soft_acc(expected_preds[test_file_index], preds[0])

    output = model.output[:, targ_class]
    grads = K.gradients(output, last_conv_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2))
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([x])
    for i in range(int(last_conv_layer.output.shape[3])):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    
    img = next(d.test_gen)[0][test_file_index]
    fig = plt.figure(figsize=(10, 5), dpi=100)
    ax = fig.add_subplot(131)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    ax.set_axis_off()
    ax.matshow(heatmap)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed = ((heatmap*0.5/np.max(heatmap) + img)) / 1.5
    ax = fig.add_subplot(132)
    ax.set_axis_off()
    ax.imshow(imshow_friendly(superimposed))
    ax.set_title('%s? %s' % (d.labels[targ_class], 'yes' if result == 1 else 'no'), fontsize=12)
    ax = fig.add_subplot(133)
    ax.set_axis_off()
    ax.imshow(imshow_friendly(img))
    fig.show()

In [ ]:
for i in range(20):
    visualize_cam(model, 'model_weights_block4_conv1_imagenet_aug_no_mixup.h5', i, datapath / 'test_difficult',
                  test_difficult_expected_preds)